# Setup

In [30]:
"""
Page setup and execution for the aerospace mechanism chatbot
Example :        
-What can you tell me about latch mechanism design failures which have occurred        
-Follow up: Which one of the sources discussed volatile spherical joint interfaces           
"""
# import databutton as db

import os
import glob
import data_import
import queries
from dotenv import load_dotenv, find_dotenv
import streamlit as st
import openai
import secrets

from IPython.display import display, Markdown
import pprint

from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)

True

In [31]:
# Check api keys
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))
# print(os.getenv('HUGGING_FACE_API_KEY'))
# print(os.getenv('LANGCHAIN_TRACING_V2'))
# print(os.getenv('LANGCHAIN_ENDPOINT'))
# print(os.getenv('LANGCHAIN_API_KEY'))
# print(os.getenv('LANGCHAIN_PROJECT'))

## Tradeable fields

In [32]:
output_level=None   # Max token limit, see section below
k=10 # Number of queries to return
search_type='similarity'    #  'mmr' or 'similarity'
temperature=0
verbose=True
chain_type='stuff'  # 'stuff' or  'map_reduce'

In [33]:
# Process some items
if output_level=="Concise":
    max_generated_tokens=50
elif output_level=="Detailed":
    max_generated_tokens=516
else:
    # max_generated_tokens = -1
    max_generated_tokens=None

# Track filtering, set history to be blank
message_id=0
filter_toggle=False # Filter sources on last answer
chat_history = []

## Vector database

### Pinecone

In [34]:
import pinecone

pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENVIRONMENT')
)
index_name = 'canopy--ams'

## Embeddings

### Openai Embeddings

In [35]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

### Voyage embeddings

## LLM

Run one of the cells below to get an llm that plugs into the prompts section.

### Proprietary

#### Openai
Models: https://platform.openai.com/docs/models/gpt-3-5

In [45]:
from langchain.chat_models import ChatOpenAI

openai.api_key = os.getenv('OPENAI_API_KEY')

# model_name='gpt-3.5-turbo-instruct'
model_name='gpt-3.5-turbo-1106' # 16,385 tokens
# model_name='gpt-4-0613'

llm = ChatOpenAI(temperature=temperature,
             model_name=model_name,
             max_tokens=max_generated_tokens,
             tags=[model_name+'-'+str(temperature)+'-'+str(max_generated_tokens)])

### Open Source

#### Ollama (local, works on mac only)

In [37]:
from langchain.llms import Ollama

model_name='llama2:latest'
llm = Ollama(model=model_name)

In [38]:
# llm("the first man on the moon was...")

#### llama.cpp (local)

This works but is really limited and difficult to scale. Recommend using the run via API in the next section.
* Try this: https://replicate.com/blog/run-llama-locally

In [39]:
# from langchain.llms import HuggingFacePipeline
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

# model_id='google/flan-t5-small'

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# # model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')
# # model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=False, device_map='auto')
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# pipeline = pipeline(
#     "text2text-generation",
#     model=model, 
    
#     tokenizer=tokenizer, 
#     max_length=128
# )

# llm = HuggingFacePipeline(pipeline=pipeline)

In [40]:
# print(llm('What is the capital of England?'))

london


#### Hugging face models (via API)

In [41]:
from langchain.llms import HuggingFaceHub

# model_id='google/flan-t5-xxl'
model_id='WizardLM/WizardLM-70B-V1.0'
# model_id='meta-llama/Llama-2-70b-chat-hf'

llm = HuggingFaceHub(repo_id=model_id, 
                     model_kwargs={"temperature": temperature}
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


# Prompts

In [43]:
prompt=['Prompt 1: What types of lubricants are to be avoided when designing space mechanisms?',
        'Prompt 2: Can you speak to what failures have occurred when using mineral oil lubricants?']

## First prompt

In [46]:
# Set up object
tags=['prompt1']
qa_model_obj=queries.QA_Model(index_name,
                    embeddings_model,
                    llm,
                    k,
                    search_type,
                    verbose,
                    filter_arg=filter_toggle)

# Generate a response using your chat model
qa_model_obj.query_docs(prompt[0],tags)
ai_response=qa_model_obj.result['answer']
references=qa_model_obj.sources[-1]

display(Markdown(prompt[0]))
display(Markdown(ai_response))
pprint.pprint(references,width=500)

Prompt 1: What types of lubricants are to be avoided when designing space mechanisms?

Perfluoropolyethers (PFPEs) and multiply alkylated cyclopentanes (MAC) are heritage lubricants used in space applications. They both have benefits and drawbacks; the main benefit being outstanding resistance to outgassing, but their tribofilm forming properties are problematic. PFPE forms iron fluourides in tribocontacts, which prevents seizure but eventually degrades the system autocatalytically. MAC on the other hand is a neat hydrocarbon, and is not generally tribochemically active. Additives are possible, but finding effective additives that are miscible and non-volatile is challenging, and few options are currently available. Therefore, these types of lubricants are to be avoided when designing space mechanisms.
SOURCES: AMS_2020.pdf, AMS_2020.pdf

[{'document_id': 'AMS_1998.pdf_31107727471-051f-4979-8b7b-ff5aafb8233d', 'page': '311', 'source': 'AMS_1998.pdf'},
 {'document_id': 'AMS_2020.pdf_2684dd10ab7-7371-4b76-9347-8e22cb98ea41', 'page': '268', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_2655d737dec-85ef-4c92-a748-5606ab9cf2bc', 'page': '265', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_308b4ceecb7-7783-48c3-9975-62fd28ebe114', 'page': '308', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_297f3e14198-e3a0-4689-989e-a3dee241f0ae', 'page': '297', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_1997.pdf_289ab65038c-470e-4a9f-b81f-bc837a90ff37', 'page': '289', 'source': 'AMS_1997.pdf'},
 {'document_id': 'AMS_2004.pdf_288186c6849-fdc1-4229-b4af-9017670e8110', 'page': '288', 'source': 'AMS_2004.pdf'},
 {'document_id': 'AMS_2020.pdf_52786cdb6fb-8d6a-444e-a0db-b1ed3e0f2602', 'page': '527', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2018.pdf_90422f64ac-3adc-48fa-adf3-44bbf647e037', 'page': 

## Second Prompt

In [10]:
# Generate a response using your chat model
tags=['prompt2_nofilter']
qa_model_obj.update_model(llm=llm,
                          filter_arg=False)

qa_model_obj.query_docs(prompt[1],
                        tags=tags)
ai_response=qa_model_obj.result['answer']
references=qa_model_obj.sources[-1]

display(Markdown(prompt[1]))
display(Markdown(ai_response))
pprint.pprint(references,width=500)

Prompt 2: Can you speak to what failures have occurred when using mineral oil lubricants?

Failures that have occurred when using mineral oil lubricants include dewetting problems, volatility issues, and degradation of the anti-wear additives. There have also been concerns about bearing lifetimes due to the use of new, unqualified ODC-free cleaning techniques. Additionally, mineral oils have wide molecular weight distributions and are susceptible to autocatalytic degradation. The lead additives have also performed poorly in vacuum bearing tests, and there have been issues with lubricant breakdown due to a chemical reaction with moisture. Finally, there have been instances of conservative temperature prediction leading to a premature loss of lubrication.
SOURCES: AMS_1998.pdf, AMS_2020.pdf, AMS_2016.pdf, AMS_1999.pdf

[{'document_id': 'AMS_1998.pdf_31107727471-051f-4979-8b7b-ff5aafb8233d', 'page': '311', 'source': 'AMS_1998.pdf'},
 {'document_id': 'AMS_2020.pdf_320e5ad8809-aac0-43e1-81c0-38b1d743aab4', 'page': '320', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_4159fe497be-d2d6-4adf-a041-0e7aed806f63', 'page': '415', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_2020.pdf_3257ab5ab61-246b-483a-9b8e-79c29ca1220c', 'page': '325', 'source': 'AMS_2020.pdf'},
 {'document_id': 'AMS_1999.pdf_174d5974ce7-270b-4189-95c5-4de163d6c894', 'page': '174', 'source': 'AMS_1999.pdf'},
 {'document_id': 'AMS_2016.pdf_9396788347-0781-4cb9-8062-b2ec3847bb4d', 'page': '93', 'source': 'AMS_2016.pdf'}]


# Langsmith

In [11]:
# from langchain.chat_models import ChatOpenAI

# llm=ChatOpenAI()
# llm.invoke("Hello world!")